In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai
import os 


In [11]:
cwd = os.getcwd()

In [39]:
hhs_region = pd.read_csv(cwd + '/raw_data/hhs.csv')

In [13]:
ref = pd.read_pickle(cwd + '/processed_data/final_data.pkl')

In [14]:
api_key = "Your Key"
openai.api_key = api_key

In [15]:
df_v = pd.read_pickle(cwd + '/processed_data/top_5_variant.pkl')

In [16]:
df_v.iloc[:, -2:] = round(df_v.iloc[:, -2:], 2)
df_v = df_v[df_v['published_date'] >= '2022-02-01'].reset_index().drop(columns = ['index'])
df_v = df_v[~df_v['variant'].isin(['BF.7', 'FD.2'])].reset_index().drop(columns = ['index'])
df_v = df_v[~df_v['variant'].isin(['B.1'])].reset_index().drop(columns = ['index'])
df_v['Week'] = pd.to_datetime(df_v['published_date']).dt.to_period('W')
df_v['Week_start'] = df_v['Week'].dt.start_time.dt.strftime('%Y-%m-%d')
df_v['Week_end'] = df_v['Week'].dt.end_time.dt.strftime('%Y-%m-%d')
df_v = df_v.drop(columns = ['published_date'])
df_v = df_v[df_v['prev_2w'] > 0.01].reset_index().drop(columns = ['index'])
df_v = df_v[df_v['Week_start'] < '2023-04-01'].reset_index().drop(columns = ['index'])

In [17]:
df_v['prev_2w'] = df_v['prev_2w']*100
df_v['prev_1w'] = df_v['prev_1w']*100
df_v['prev_1w'] = round(df_v['prev_1w'],0)
df_v['prev_2w'] = round(df_v['prev_2w'],0)
df_v['prev_1w'] = df_v['prev_1w'].astype(str) + '%'
df_v['prev_2w'] = df_v['prev_2w'].astype(str) + '%'


In [18]:
for index, row in df_v.iloc[:, ].iterrows():
    prompt =  "Describe the trend of new COVID-19 emerging variant proportion in recent weeks" + \
    str(df_v.loc[index, ['prev_2w', 'prev_1w']].to_numpy())
    
    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      temperature = 1,
      max_tokens = 100,
      seed=1,
      messages = [
        {"role": "system", 
         "content": "You are a helpful assistant for summarizing the key Trends and comment on Rate of Change for new emerging COVID-19 variant trend in a concise written format (one sentence)."},
        {"role": "user", "content" : prompt}
      ]
    )
    
    df_v.loc[index, 'variant_gpt_trend'] = completion.choices[0].message.content
#     print(index)    

In [22]:
df_variant_des = pd.DataFrame(df_v['variant'].unique())
df_variant_des = df_variant_des.rename(columns = {0 : 'variant'})

In [23]:
df_variant_des.loc[0, 'transmission'] = "The new emerging variant is more transmissible than previous sublineages."
df_variant_des.loc[1, 'transmission'] = "The new emerging variant has high growth advantages over other sublineages."
df_variant_des.loc[2, 'transmission'] = "The new emerging variant has high growth advantages over other sublineages."
df_variant_des.loc[3, 'transmission'] = "The new emerging variant is more transmissible than previous sublineages."
df_variant_des.loc[4, 'transmission'] = "The new emerging variant has a increased impact on transmissibility. ."

df_variant_des.loc[0, 'immunity'] = "No evidence in immunity escape."
df_variant_des.loc[1, 'immunity'] = "Increased immunity escape, may cause an overall increase in COVID-19 cases."
df_variant_des.loc[2, 'immunity'] = "Increased immunity escape, may cause an overall increase in COVID-19 cases."
df_variant_des.loc[3, 'immunity'] = "No reported increase in immunity escape."
df_variant_des.loc[4, 'immunity'] = "Increased impact on immunity escape."

df_variant_des.loc[0, 'severity'] = "No reported difference in severity."
df_variant_des.loc[1, 'severity'] = "No significant increase in infection severity."
df_variant_des.loc[2, 'severity'] = "No significant increase in infection severity."
df_variant_des.loc[3, 'severity'] = "No reported increase in disease severity."
df_variant_des.loc[4, 'severity'] = "No evidence on impact on severity."


In [24]:
df_v = df_v.drop_duplicates(['usa_or_hhsregion', 'variant', 'Week', 'Week_start', 'Week_end']).reset_index().drop(columns = ['index'])
df_v = df_v.merge(df_variant_des)
index = df_v[(df_v['usa_or_hhsregion'] == '1')&(df_v['variant'] == 'BA.2')].index
df_v.loc[index, 'BA.1 Week'] = np.arange(27)
for region in df_v['usa_or_hhsregion'].unique():
    for variant in df_v['variant'].unique():
        index = df_v[(df_v['usa_or_hhsregion'] == region)&(df_v['variant'] == variant)].index
        length = len(df_v[(df_v['usa_or_hhsregion'] == region)&(df_v['variant'] == variant)])
        
        df_v.loc[index, variant + ' Week'] = np.arange(length)
        

In [25]:
df_v = df_v.drop(columns = ['BA.1 Week'])
df_v['BA.2 Week'] = df_v['BA.2 Week'].replace(0, np.nan)
df_v = df_v.dropna(subset = ['BA.2 Week', 'BA.4 Week', 'BA.5 Week', 'BQ.1 Week', 'XBB Week'], how='all')
df_v = df_v.reset_index().drop(columns = ['index'])
df_v['BA.4 Week'] = df_v['BA.4 Week'].replace(0, np.nan)
df_v = df_v.dropna(subset = ['BA.2 Week', 'BA.4 Week', 'BA.5 Week', 'BQ.1 Week', 'XBB Week'], how='all')
df_v = df_v.reset_index().drop(columns = ['index'])

In [40]:
df_8_weeks = df_v[df_v['BA.2 Week'] <= 8].append(df_v[df_v['BA.5 Week'] <= 8]).append(df_v[df_v['BQ.1 Week'] <= 8])
df_8_weeks['Week_since_detected'] = df_8_weeks['BA.2 Week'].fillna(0)\
+ df_8_weeks['BA.5 Week'].fillna(0) + df_8_weeks['BQ.1 Week'].fillna(0)
df_8_weeks = df_8_weeks.drop(columns = ['BA.2 Week', 'BA.4 Week', 'BA.5 Week',
                          'BQ.1 Week', 'XBB Week'])
df_8_weeks['Week_since_reported'] = 'The new variant has been reported by CDC for ' + \
df_8_weeks['Week_since_detected'].astype(str).str[:1] + ' Weeks'
df_8_weeks = df_8_weeks.drop(columns = ['Week_since_detected'])
df_8_weeks['usa_or_hhsregion'] = df_8_weeks['usa_or_hhsregion'].astype(int)
df_8_weeks = df_8_weeks.rename(columns = {'usa_or_hhsregion' : 'HHS_region'})

/var/folders/fg/qtvw7sp54r91rm2skfzzb8ww0000gn/T/ipykernel_33075/2381070107.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_8_weeks = df_v[df_v['BA.2 Week'] <= 8].append(df_v[df_v['BA.5 Week'] <= 8]).append(df_v[df_v['BQ.1 Week'] <= 8])
/var/folders/fg/qtvw7sp54r91rm2skfzzb8ww0000gn/T/ipykernel_33075/2381070107.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_8_weeks = df_v[df_v['BA.2 Week'] <= 8].append(df_v[df_v['BA.5 Week'] <= 8]).append(df_v[df_v['BQ.1 Week'] <= 8])


In [41]:
hhs_region = hhs_region.rename(columns = {'state_or_territory' : 'state_name'})
hhs_region = hhs_region[hhs_region['state_name'].isin(ref['state_name'])]
hhs_region = hhs_region.reset_index().drop(columns = ['index'])
hhs_region = hhs_region.drop(columns = ['region', 'regional_office']).rename(columns = {'region_number' : 'HHS_region'})
df_8_weeks = df_8_weeks.merge(hhs_region)

In [42]:
df_8_weeks = df_8_weeks.drop(columns = ['prev_2w', 'prev_1w', 'HHS_region'])

In [43]:
df_8_weeks = df_8_weeks.drop(columns = 'Unnamed: 0')

In [44]:
df_8_weeks.to_pickle(cwd + '/processed_data/genomic_prompt.pkl')